In [0]:
# Importando módulo de seguimiento
import sys
import importlib
sys.path.append('/Workspace/Users/luisfdiaz@bcp.com.pe/Modulo de Seguimiento/')

import SegScore as rmm
importlib.reload(rmm)

# Librerias y fuciones generales
from pyspark.sql.functions import date_format, expr, to_date, date_sub, add_months, col, when, coalesce, trim, broadcast, avg, max, min, lit, concat, window, round as colround, upper, abs as sparkabs,greatest
from pyspark.sql import functions as F
from pyspark import StorageLevel
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.sql.window import Window

In [0]:
base_ivan = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_base_app_hip')
base_seguimiento = spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_univ_pre_driver_app_hip')

In [0]:
base_0 = base_ivan.withColumn(
    'TIPO_FONDEO',
    F.when(base_ivan['TIPCREDITO'].isin('TH01', 'TH02', 'HICC', 'RHIP', 'RREF', 'HICB', 'HIFS', 'HIBS', 'HIBK', 'HIPA', 'HIPD', 'HIPO', 'HIPP', 'HIPT', 'HIPR', 'HCPO', 'HITR', 'HTIV'), 'FONDEO BCP')
    .when(base_ivan['TIPCREDITO'].isin('GAHI', 'GAHP', 'GAHR', 'GAPC'), 'GAHI')
    .when(base_ivan['TIPCREDITO'].isin('HITP', 'HMIV', 'HVIV', 'HSIV', 'HBIV'), 'FONDEO MVIV')
    .otherwise('OTROS')
)

gremio=base_seguimiento.filter((base_seguimiento['CODMES'] >= 201901) & (base_seguimiento['FLG_GREMIO_F'] == 1)) \
               .select('CODCLAVEOPECTA', 'FLG_GREMIO_F')

missingPD = base_seguimiento.filter((base_seguimiento['CODMES'] >= 201901) & (F.isnull(base_seguimiento['PD_ESTADOS']))) \
              .select('CODCLAVEOPECTA')

In [0]:
hm_adm_hip_ahlo=spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hipahlo')
hm_adm_hip_trad=spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hiptrad')
hm_adm_hip_pyme=spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_hippyme')
hm_adm_hip_gahi=spark.table('catalog_lhcl_prod_bcp.bcp_edv_fabseg.t39290_hm_app_gahi')

hm_adm_hip_ahlo = hm_adm_hip_ahlo.drop("PD_2024Q4vf")
hm_adm_hip_ahlo = hm_adm_hip_ahlo.withColumn("PD_2024Q4vf", when(col("CODMES") >=202307,col( "PD_2024Q4v2")).otherwise(col("PD_2024Q4")))

hm_adm_hip_ahlo = hm_adm_hip_ahlo.withColumn("MISS_PD", when(col("PD_2024Q4vf").isNull(), 1).otherwise(0))

base_ah_lo = hm_adm_hip_ahlo.filter(hm_adm_hip_ahlo['CODMES'] >= 201901) \
                            .select('CODCLAVEOPECTA', 'DEF_120_BLOQ_REF_0', 'MISS_PD', 'CAMPANIA_AGRUPADA')
base_hip_trad = hm_adm_hip_trad.filter(hm_adm_hip_trad['CODMES'] >= 201901) \
                            .select('CODCLAVEOPECTA', 'DEF_120_BLOQ_REF_0', 'MISS_PD', 'CAMPANIA_AGRUPADA')
base_hip_pyme = hm_adm_hip_pyme.filter(hm_adm_hip_pyme['CODMES'] >= 201901) \
                            .select('CODCLAVEOPECTA', 'DEF_120_BLOQ_REF_0', 'MISS_PD', 'CAMPANIA_AGRUPADA')
base_gahi = hm_adm_hip_gahi.filter(hm_adm_hip_gahi['CODMES'] >= 201901) \
                            .select('CODCLAVEOPECTA', 'DEF_120_BLOQ_REF_0', 'MISS_PD', 'CAMPANIA_AGRUPADA')

In [0]:
hm_universo_app_hip = base_0.alias("A") \
    .join(gremio.alias("E"), F.col("A.CODCLAVEOPECTA") == F.col("E.CODCLAVEOPECTA"), "left") \
    .join(missingPD.alias("G"), F.col("A.CODCLAVEOPECTA") == F.col("G.CODCLAVEOPECTA"), "left") \
    .join(base_hip_trad.alias("H1"), F.col("A.CODCLAVEOPECTA") == F.col("H1.CODCLAVEOPECTA"), "left") \
    .join(base_ah_lo.alias("H2"), F.col("A.CODCLAVEOPECTA") == F.col("H2.CODCLAVEOPECTA"), "left") \
    .join(base_hip_pyme.alias("H3"), F.col("A.CODCLAVEOPECTA") == F.col("H3.CODCLAVEOPECTA"), "left") \
    .join(base_gahi.alias("H4"), F.col("A.CODCLAVEOPECTA") == F.col("H4.CODCLAVEOPECTA"), "left") \
    .filter(F.col("A.CODMES") >= 201901) \
    .select(
        F.col("A.*"),
        F.when(F.col("H1.CODCLAVEOPECTA").isNotNull() & (F.col("H1.DEF_120_BLOQ_REF_0") == 0) & (F.col("H1.MISS_PD") == 0), '1. Hip Tradicional')
        .when((F.col("A.CODMES") <= 202407) & F.col("H2.CODCLAVEOPECTA").isNotNull() & (F.col("H2.DEF_120_BLOQ_REF_0") == 0) & (F.col("H2.MISS_PD") == 0), '2. Hip Ahorro local')
        .when((F.col("A.CODMES") > 202407) & F.col("H2.CODCLAVEOPECTA").isNotNull() & (F.col("H2.DEF_120_BLOQ_REF_0") == 0), '2. Hip Ahorro local')
        .when(F.col("H3.CODCLAVEOPECTA").isNotNull() & (F.col("H3.DEF_120_BLOQ_REF_0") == 0) & (F.col("H3.MISS_PD") == 0), '3. Hip Pyme')
        .when(F.col("H4.CODCLAVEOPECTA").isNotNull() & (F.col("H4.DEF_120_BLOQ_REF_0") == 0) & (F.col("H4.MISS_PD") == 0), '4. Gahi')
        .otherwise('Sin Seguimiento').alias("flag_seguimiento"),
        F.coalesce(F.col("H1.DEF_120_BLOQ_REF_0"), F.col("H2.DEF_120_BLOQ_REF_0"), F.col("H3.DEF_120_BLOQ_REF_0"), F.col("H4.DEF_120_BLOQ_REF_0")).alias("DEF_120_BLOQ_REF_0"),
        F.coalesce(F.col("H1.MISS_PD"), F.col("H2.MISS_PD"), F.col("H3.MISS_PD"), F.col("H4.MISS_PD")).alias("MISS_PD"),
        F.when(F.col("E.CODCLAVEOPECTA").isNull(), 0).otherwise(1).alias("flag_GREMIO"),
        # F.when(F.col("F.CODCLAVEOPECTA").isNull(), 0).otherwise(1).alias("flag_DUP_SOLI"),
        F.when(F.col("G.CODCLAVEOPECTA").isNull(), 0).otherwise(1).alias("flag_MISS_PD")
    )

In [0]:
hm_universo_app_hip = hm_universo_app_hip.withColumn(
    'PRODUCTO_NEW',
    F.when(F.col('MISS_PD') == 1, '4. Missing PD')
    .when(F.col('TIPCREDITO').isin('HVIC', 'HVIM', 'HBIC', 'HSIC', 'HMIC', 'HMIM', 'HREF'), '1. Tramo Concesional')
    # .when(F.col('flag_DUP_SOLI') == 1, '3. Duplicado')
    .when(F.col('flag_GREMIO') == 1, '2. Gremio')
    .when((F.col('TIPO_FONDEO') == 'FONDEO MVIV') & (F.col('CAMPANIA_AGRUPADA') == 'Ahorro Local'), '6. Ahorro Local Fondeo Mivivienda')
    .when((F.col('TIPO_FONDEO') == 'FONDEO BCP') & (F.col('CAMPANIA_AGRUPADA') == 'Pyme'), '7. Pyme Fondeo BCP')
    .when((F.col('TIPO_FONDEO') == 'FONDEO BCP') & (F.col('CAMPANIA_AGRUPADA') == 'Tradicional'), '8. Tradicional Fondeo BCP')
    .when((F.col('TIPO_FONDEO') == 'FONDEO BCP') & (F.col('CAMPANIA_AGRUPADA') == 'Ahorro Local'), '9. Ahorro Local Fondeo BCP')
    .when((F.col('TIPO_FONDEO') == 'FONDEO MVIV') & (F.col('CAMPANIA_AGRUPADA') == 'Pyme'), '91. Pyme Fondeo Mivivienda')
    .when((F.col('TIPO_FONDEO') == 'FONDEO MVIV') & (F.col('CAMPANIA_AGRUPADA') == 'Tradicional'), '92. Tradicional Fondeo Mivivienda')
    .when((F.col('TIPO_FONDEO') == 'FONDEO MVIV') & F.col('CAMPANIA_AGRUPADA').isNull(), '93. Ahorro Local Observado')
    .when((F.col('TIPO_FONDEO') == 'FONDEO BCP') & F.col('CAMPANIA_AGRUPADA').isNull(), '94. Tradicional Observado')
    .when((F.col('TIPO_FONDEO') == 'GAHI') & F.col('TIPCREDITO').isin('GAHI', 'GAHP', 'GAHR', 'GAPC'), '5. Garantia Hipotecaria')
    .otherwise('OTROS')
)

In [0]:
from pyspark.sql import functions as F

# Filtrar por CODMES entre 202411, 202412 y 202501
filtered_df = hm_universo_app_hip.filter(F.col("CODMES").isin([202501, 202502, 202503]))
filtered_df = filtered_df.withColumn("CODMES", col("CODMES").cast("integer"))

# Agrupar por PRODUCTO_NEW y CODMES, contar y sumar MTOAPROBADO_SOLES
grouped_df = filtered_df.groupBy("PRODUCTO_NEW", "CODMES").agg(
    F.count("*").alias("Cuenta"),
    (F.sum("MTOAPROBADO_SOLES") / 1_000_000).alias("Monto")
)

# Crear tabla cruzada (crosstab)
crosstab_df = grouped_df.groupBy("PRODUCTO_NEW").pivot("CODMES").agg(
    F.first("Cuenta").alias("Cuenta"),
    F.first("Monto").alias("Monto")
)

In [0]:
vista_1=crosstab_df.toPandas()
vista_1.sort_values(by='PRODUCTO_NEW')

In [0]:
from pyspark.sql import functions as F

# Filtrar por CODMES entre 202411, 202412 y 202501
filtered_df = hm_universo_app_hip.filter(F.col("CODMES").isin([202411, 202412, 202501]))

# Agrupar por PRODUCTO_NEW y CODMES, contar y sumar MTOAPROBADO_SOLES
grouped_df = filtered_df.groupBy("PRODUCTO_NEW").agg(
    (F.count("*")/3).alias("Cuenta"),
    (F.sum("MTOAPROBADO_SOLES") / 3_000_000).alias("Monto")
)

# Crear tabla cruzada (crosstab)
crosstab_df = grouped_df.groupBy("PRODUCTO_NEW").agg(
    F.first("Cuenta").alias("Cuenta"),
    F.first("Monto").alias("Monto")
)

vista_2=crosstab_df.toPandas()
vista_2.sort_values(by='PRODUCTO_NEW')